# Incident Report Deduplication System

This notebook processes the `Case Log.xlsx` file and creates a deduplicated CSV with consolidated incident reports and their root causes.

## Approach:
1. Load the source Excel file
2. Clean and preprocess text (remove numbers, IDs, timestamps)
3. Use spaCy NLP for phrase-level similarity analysis
4. Compare incidents and solutions using 90% similarity threshold
5. Generate concise incident reports
6. Save to `processed-data/consolidated_incidents.csv`

## Step 1: Import Required Libraries

In [4]:
import pandas as pd
import numpy as np
import spacy
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Load spaCy model (using medium model for better accuracy)
try:
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("Downloading spaCy model...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_md"])
    nlp = spacy.load("en_core_web_md")

print("Libraries loaded successfully!")
print(f"spaCy model: {nlp.meta['name']}")

Libraries loaded successfully!
spaCy model: core_web_md


## Step 2: Load the Source Data

In [5]:
# Define paths
source_file = Path("source-data/Case Log.xlsx")
output_file = Path("processed-data/consolidated_incidents.csv")

# Create output directory if it doesn't exist
output_file.parent.mkdir(exist_ok=True)

# Load the Excel file
df = pd.read_excel(source_file)

print(f"Loaded {len(df)} rows from {source_file}")
print(f"\nColumns in the file: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Loaded 323 rows from source-data\Case Log.xlsx

Columns in the file: ['Module', 'Mode', 'EDI?', 'TIMESTAMP', 'Alert / Email', 'Problem Statements', 'Solution', 'SOP']

First few rows:


,Module,Mode,EDI?,TIMESTAMP,Alert / Email,Problem Statements,Solution,SOP
0,EDI/API,Call,Yes,2025-02-25 07:34:00,Call ALR-360601 | EDI/API Data Mismatch on HLC...,Time zone drift caused eventTime to serialize ...,Normalized eventTime to port timezone and adde...,NaN
1,EDI/API,SMS,Yes,2025-06-30 06:08:00,Alert: SMS TCK-265455 | Issue: Spike in DLQ me...,Spike in DLQ messages after routine maintenanc...,Corrected cron schedule to UTC; added mutual e...,EDI: Spike in DLQ messages after routine maint...
2,Vessel,SMS,Yes,2025-07-04 06:29:00,Alert: SMS TCK-936729 | Issue: ANSI X12 301 in...,ANSI X12 301 inconsistency for MV SILVER CURRE...,Enabled conflict resolution preferring max(eve...,VSL: Duplicate EDI De-duplication
3,EDI/API,Email,Yes,2025-06-04 00:02:00,Subject: Email ALR-535708 | EDI Parsing Issue ...,EDIFACT CODECO duplicate detected with conflic...,"Added de-duplication keyed by (controlNumber, ...",NaN
4,Container Report,SMS,No,2025-01-15 20:18:00,Alert: SMS TCK-142185 | Issue: Customs flag to...,Customs flag toggled incorrectly for EMCU16695...,Manual verification per CNTR-07 (Yard Location...,NaN


## Step 3: Text Preprocessing Functions

Create functions to clean and normalize text by removing numbers, IDs, timestamps, and other variable details while preserving the core problem description.

In [9]:
# Caches to avoid re-processing
_doc_cache = {}
_phrase_cache = {}
_sim_cache = {}


def _get_doc(text: str):
    if text in _doc_cache:
        return _doc_cache[text]
    d = nlp(text)
    _doc_cache[text] = d
    return d


def clean_text(text):
    """
    Clean text by removing numbers, IDs, timestamps, and other variable details
    while preserving the core problem description.
    """
    if pd.isna(text) or text == "":
        return ""
    
    text = str(text)
    
    # Remove common patterns that vary between incidents
    # Remove timestamps (various formats)
    text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '[DATE]', text)  # 2025-10-18
    text = re.sub(r'\b\d{1,2}[:/\-]\d{1,2}[:/\-]\d{2,4}\b', '[DATE]', text)  # 10/18/2025, 18-10-25
    text = re.sub(r'\b\d{1,2}:\d{2}(:\d{2})?(\s?[AaPp][Mm])?\b', '[TIME]', text)  # 10:30, 10:30 AM
    
    # Remove IDs and numbers (but keep words)
    text = re.sub(r'\b[A-Z]{2,}\d+\b', '[ID]', text)  # INC123, REQ456, etc.
    text = re.sub(r'\b\d{5,}\b', '[ID]', text)  # Long numbers (IDs)
    text = re.sub(r'#\d+', '[ID]', text)  # Ticket numbers like #12345
    
    # Remove standalone numbers but keep those within words
    text = re.sub(r'\s\d+\s', ' ', text)
    text = re.sub(r'^\d+\s', '', text)
    text = re.sub(r'\s\d+$', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text.lower()


def get_phrases(text):
    """
    Split text into meaningful phrases using spaCy's sentence detection.
    Cached by cleaned text to avoid recomputation.
    """
    if not text or text == "":
        return []
    
    if text in _phrase_cache:
        return _phrase_cache[text]
    
    doc = _get_doc(text)
    phrases = []
    
    # Get sentences as phrases
    for sent in doc.sents:
        phrase = sent.text.strip()
        if len(phrase.split()) >= 2:  # Only keep phrases with 2+ words
            phrases.append(phrase)
    
    _phrase_cache[text] = phrases
    return phrases


def calculate_phrase_similarity(text1, text2):
    """
    Calculate similarity between two texts using phrase-level analysis.
    Returns the average similarity score across all phrase pairs.
    Uses caching to avoid re-computation for the same pair.
    """
    # Clean both texts
    clean1 = clean_text(text1)
    clean2 = clean_text(text2)
    
    if not clean1 or not clean2:
        return 0.0
    
    # Use a canonical key for pairwise cache (order-independent)
    pair_key = tuple(sorted([clean1, clean2]))
    if pair_key in _sim_cache:
        return _sim_cache[pair_key]
    
    # Get phrases from both texts (from cache if available)
    phrases1 = get_phrases(clean1)
    phrases2 = get_phrases(clean2)
    
    if not phrases1 or not phrases2:
        # Fallback to direct document similarity if no phrases found
        doc1 = _get_doc(clean1)
        doc2 = _get_doc(clean2)
        sim = doc1.similarity(doc2)
        _sim_cache[pair_key] = sim
        return sim
    
    # Calculate similarity for all phrase pairs
    similarities = []
    for p1 in phrases1:
        phrase_sims = []
        doc1 = _get_doc(p1)
        for p2 in phrases2:
            doc2 = _get_doc(p2)
            sim = doc1.similarity(doc2)
            phrase_sims.append(sim)
        # For each phrase in text1, take the max similarity with any phrase in text2
        if phrase_sims:
            similarities.append(max(phrase_sims))
    
    sim_avg = np.mean(similarities) if similarities else 0.0
    _sim_cache[pair_key] = sim_avg
    return sim_avg


def generate_concise_text(text):
    """
    Generate a concise version of the text by extracting key information.
    """
    if pd.isna(text) or text == "":
        return ""
    
    text = str(text).strip()
    
    # Remove excessive details and keep the core message
    doc = _get_doc(text)
    
    # Extract key sentences (up to 2 sentences)
    sentences = [sent.text.strip() for sent in doc.sents]
    
    if len(sentences) <= 2:
        return text
    
    # Keep first two sentences as a concise summary
    concise = sentences[0]
    if len(sentences) > 1:
        concise += " " + sentences[1]
    
    return concise.strip()


print("Text preprocessing + caching functions created successfully!")

Text preprocessing + caching functions created successfully!


## Step 4: Deduplication Algorithm

Implement the core deduplication logic that:
1. Iterates through each incident in the source data
2. Compares with existing entries in the consolidated CSV
3. Uses 90% similarity threshold for both incidents and solutions
4. Generates concise versions of the text

In [7]:
def deduplicate_incidents(source_df, alert_col='Alert/Email', problem_col='Problem Statement', similarity_threshold=0.90):
    """
    Main deduplication function that processes incidents one by one.
    
    Parameters:
    - source_df: Source DataFrame with incident data
    - alert_col: Column name for incident alerts/emails
    - problem_col: Column name for problem statements
    - similarity_threshold: Threshold for considering incidents as duplicates (default 0.90)
    
    Returns:
    - DataFrame with deduplicated incidents
    """
    # Initialize empty list to store consolidated incidents
    consolidated = []
    
    # Track statistics
    total_processed = 0
    duplicates_found = 0
    unique_incidents = 0
    
    print(f"Starting deduplication process...")
    print(f"Similarity threshold: {similarity_threshold * 100}%\n")
    
    # Iterate through each row in the source data
    for idx, row in source_df.iterrows():
        total_processed += 1
        
        # Get current incident and problem statement
        current_alert = row[alert_col]
        current_problem = row[problem_col]
        
        # Skip rows with empty/null values
        if pd.isna(current_alert) or pd.isna(current_problem) or \
           str(current_alert).strip() == "" or str(current_problem).strip() == "":
            continue
        
        # Generate concise versions
        concise_alert = generate_concise_text(current_alert)
        concise_problem = generate_concise_text(current_problem)
        
        # Check if this is a duplicate
        is_duplicate = False
        
        for existing in consolidated:
            # Compare incidents using phrase-level similarity
            incident_similarity = calculate_phrase_similarity(current_alert, existing['original_alert'])
            
            if incident_similarity >= similarity_threshold:
                # Found a similar incident, now check if the solution is also similar
                solution_similarity = calculate_phrase_similarity(current_problem, existing['original_problem'])
                
                if solution_similarity >= similarity_threshold:
                    # Both incident and solution are similar - it's a duplicate
                    is_duplicate = True
                    duplicates_found += 1
                    if total_processed % 10 == 0:
                        print(f"Row {total_processed}: Duplicate found (Incident: {incident_similarity:.2%}, Solution: {solution_similarity:.2%})")
                    break
                else:
                    # Incident is similar but solution is different - treat as separate incident
                    # Continue checking other entries
                    continue
        
        if not is_duplicate:
            # Add as new unique incident
            consolidated.append({
                'Incident_Report': concise_alert,
                'Root_Cause': concise_problem,
                'original_alert': current_alert,  # Keep for comparison
                'original_problem': current_problem  # Keep for comparison
            })
            unique_incidents += 1
            if total_processed % 10 == 0:
                print(f"Row {total_processed}: New unique incident added")
    
    # Create final DataFrame (without the original columns used for comparison)
    result_df = pd.DataFrame([
        {'Incident_Report': item['Incident_Report'], 'Root_Cause': item['Root_Cause']}
        for item in consolidated
    ])
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print(f"DEDUPLICATION COMPLETE")
    print(f"{'='*60}")
    print(f"Total rows processed: {total_processed}")
    print(f"Duplicates found: {duplicates_found}")
    print(f"Unique incidents: {unique_incidents}")
    print(f"Reduction: {(duplicates_found/total_processed*100) if total_processed > 0 else 0:.1f}%")
    print(f"{'='*60}\n")
    
    return result_df


print("Deduplication function created successfully!")

Deduplication function created successfully!


## Step 5: Run the Deduplication Process

Execute the deduplication algorithm on the source data and save the results.

In [10]:
# Run the deduplication process with correct column names
# The file shows columns: 'Alert / Email' and 'Problem Statements'
consolidated_df = deduplicate_incidents(
    df,
    alert_col='Alert / Email',
    problem_col='Problem Statements',
    similarity_threshold=0.90
)

# Save to CSV
consolidated_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"✓ Consolidated incidents saved to: {output_file}")
print(f"\nPreview of the consolidated data:")
consolidated_df.head(10)

Starting deduplication process...
Similarity threshold: 90.0%

Row 10: New unique incident added
Row 20: New unique incident added
Row 30: New unique incident added
Row 40: Duplicate found (Incident: 97.17%, Solution: 96.54%)
Row 50: Duplicate found (Incident: 99.29%, Solution: 99.38%)
Row 60: Duplicate found (Incident: 92.22%, Solution: 100.00%)
Row 70: New unique incident added
Row 80: Duplicate found (Incident: 100.00%, Solution: 98.23%)
Row 90: Duplicate found (Incident: 99.14%, Solution: 98.35%)
Row 100: New unique incident added
Row 110: Duplicate found (Incident: 99.53%, Solution: 98.87%)
Row 120: New unique incident added
Row 130: Duplicate found (Incident: 99.12%, Solution: 99.38%)
Row 140: Duplicate found (Incident: 94.68%, Solution: 91.51%)
Row 150: New unique incident added
Row 160: Duplicate found (Incident: 92.38%, Solution: 98.47%)
Row 170: New unique incident added
Row 180: Duplicate found (Incident: 90.17%, Solution: 97.45%)
Row 190: Duplicate found (Incident: 91.78%, 

,Incident_Report,Root_Cause
0,Call ALR-360601 | EDI/API Data Mismatch on HLC...,Time zone drift caused eventTime to serialize ...
1,Alert: SMS TCK-265455 | Issue: Spike in DLQ me...,Spike in DLQ messages after routine maintenanc...
2,Alert: SMS TCK-936729 | Issue: ANSI X12 301 in...,ANSI X12 301 inconsistency for MV SILVER CURRE...
3,Subject: Email ALR-535708 | EDI Parsing Issue ...,EDIFACT CODECO duplicate detected with conflic...
4,Alert: SMS TCK-142185 | Issue: Customs flag to...,Customs flag toggled incorrectly for EMCU16695...
5,Subject: Email ALR-881385 | Vessel MV STELLAR ...,Schedule API intermittently returns 401 for MV...
6,Alert: SMS INC-782713 | Issue: Field mapping m...,Field mapping mismatch across systems for cont...
7,Call ALR-717476 | Container MAEU0713758 except...,Discrepancy between customer portal and TOS fo...
8,Alert: SMS INC-125434 | Issue: ETA/ETB mismatc...,ETA/ETB mismatch for MV PACIFIC GLORY (IMO 630...
9,Subject: Email INC-619806 | API Failure on /bo...,OAuth token rejection spikes on '/bookings/v1'...


## Step 6: Verify Results and Test Similarity Function

Let's test the similarity function with a few examples to ensure it's working correctly.

In [11]:
# Test cases to verify the similarity function
print("Testing Similarity Function")
print("="*60)

# Test 1: Similar incidents with different IDs
text1 = "Database connection failed for order #12345 at 10:30 AM"
text2 = "Database connection failed for order #67890 at 2:45 PM"
sim1 = calculate_phrase_similarity(text1, text2)
print(f"\nTest 1 - Similar incidents with different IDs:")
print(f"Text 1: {text1}")
print(f"Text 2: {text2}")
print(f"Similarity: {sim1:.2%} {'✓ MATCH' if sim1 >= 0.90 else '✗ NO MATCH'}")

# Test 2: Different incidents
text3 = "Database connection failed"
text4 = "User authentication error occurred"
sim2 = calculate_phrase_similarity(text3, text4)
print(f"\nTest 2 - Different incidents:")
print(f"Text 1: {text3}")
print(f"Text 2: {text4}")
print(f"Similarity: {sim2:.2%} {'✓ MATCH' if sim2 >= 0.90 else '✗ NO MATCH'}")

# Test 3: Nearly identical
text5 = "Server timeout error in production environment"
text6 = "Server timeout error in production system"
sim3 = calculate_phrase_similarity(text5, text6)
print(f"\nTest 3 - Nearly identical:")
print(f"Text 1: {text5}")
print(f"Text 2: {text6}")
print(f"Similarity: {sim3:.2%} {'✓ MATCH' if sim3 >= 0.90 else '✗ NO MATCH'}")

print("\n" + "="*60)

Testing Similarity Function

Test 1 - Similar incidents with different IDs:
Text 1: Database connection failed for order #12345 at 10:30 AM
Text 2: Database connection failed for order #67890 at 2:45 PM
Similarity: 100.00% ✓ MATCH

Test 2 - Different incidents:
Text 1: Database connection failed
Text 2: User authentication error occurred
Similarity: 61.39% ✗ NO MATCH

Test 3 - Nearly identical:
Text 1: Server timeout error in production environment
Text 2: Server timeout error in production system
Similarity: 100.00% ✓ MATCH



## Step 7: Final Statistics and Summary

In [12]:
print("FINAL SUMMARY")
print("="*60)
print(f"Source file: {source_file}")
print(f"Output file: {output_file}")
print(f"\nOriginal incidents: {len(df)}")
print(f"Consolidated incidents: {len(consolidated_df)}")
print(f"Duplicates removed: {len(df) - len(consolidated_df)}")
print(f"Reduction rate: {((len(df) - len(consolidated_df)) / len(df) * 100):.1f}%")
print("="*60)

# Display sample of consolidated data
print("\nSample of Consolidated Data:")
print(consolidated_df.to_string(max_rows=5, max_colwidth=80))

FINAL SUMMARY
Source file: source-data\Case Log.xlsx
Output file: processed-data\consolidated_incidents.csv

Original incidents: 323
Consolidated incidents: 89
Duplicates removed: 234
Reduction rate: 72.4%

Sample of Consolidated Data:
                                                                    Incident_Report                                                                       Root_Cause
0   Call ALR-360601 | EDI/API Data Mismatch on HLCU5962669 at PSA Keppel | Issue...  Time zone drift caused eventTime to serialize in UTC+0 for partner Partner-E...
1   Alert: SMS TCK-265455 | Issue: Spike in DLQ messages after routine maintenan...  Spike in DLQ messages after routine maintenance; consumer group lag increase...
..                                                                              ...                                                                              ...
87  SMS INC-108587 | Issue: EDI message REF-COP-0099 received but no acknowledgm...  EDI message REF-COP